# Generate IL expert data for behavior cloning
Given a number of historical scenarios containing real-time price and solar PV generation data, run MILP on each scenario to get the optimal solutions of load management. Then, extract the MILP expert's state-action dataset.

In [ ]:
using DataFrames, CSV
using BSON: @save, @load
using BSON, NPZ
using TOML
using JuMP
using Revise, HEMS

ENV["COLUMNS"] = 2000;

## Read raw scenarios data and home configuration

In [ ]:
root_dir = "../../"

In [ ]:
# home configuration
home_dir = joinpath(root_dir, "data/home/")
h = read_home_config(joinpath(home_dir, "homeB.toml"))
;

In [ ]:
scenario_dir = joinpath(root_dir, "data/learning/scenarios/")

In [ ]:
test_ss = BSON.load(joinpath(scenario_dir, "2021.bson"))[:ss] .|> DataFrame;

In [ ]:
# three years' data: 2019 and 2020 for train and 2021 for test 
train_ss = [BSON.load(joinpath(scenario_dir, "2019.bson"))[:ss] .|> DataFrame;
    BSON.load(joinpath(scenario_dir, "2020.bson"))[:ss] .|> DataFrame]
test_ss = BSON.load(joinpath(scenario_dir, "2021.bson"))[:ss] .|> DataFrame;

In [ ]:
length(train_ss)

## Optimize each scenario with the MILP expert

In [ ]:
IL_data_dir = joinpath(root_dir, "data/learning/IL_homeB")

In [ ]:
# run_expert!(test_ss[1:3], h; out_bson = joinpath(IL_data_dir, "test_ss_del.bson"), report_progress=true)

In [ ]:
run_expert!(test_ss, h; out_bson = joinpath(IL_data_dir, "test_ss.bson"), report_progress=true)
;

In [ ]:
# also measure the training time
@time begin
    run_expert!(train_ss, h; out_bson = joinpath(IL_data_dir, "train_ss.bson"), report_progress=true)
end
;

In [ ]:
# check the first result
first(first(train_ss), 3)

## Build state-action dataset as expert demonstrations
- Extract X-y pairs from the optimized scenarios.\
- Write into the specific `out_bson` file.

In [ ]:
test_xy = build_expert_dataset(test_ss, h; out_bson = joinpath(IL_data_dir, "test_xy.bson"), H=1)
@time begin
    train_xy = build_expert_dataset(train_ss, h; out_bson = joinpath(IL_data_dir, "train_xy.bson"), H=1)
end
;

In [ ]:
# full ones
test_xy = build_expert_dataset(test_ss, h; out_bson = joinpath(IL_data_dir, "test_xy_full.bson"), full=true)
@time begin
    train_xy = build_expert_dataset(train_ss, h; out_bson = joinpath(IL_data_dir, "train_xy_full.bson"), full=true)
end
;